# Initialize Classes

Initialize classes and libraries that maybe used in the script

In [ ]:
import requests, json, logging, time, urllib
import pandas as pd
from io import StringIO
import io as stringIOModule
from datetime import datetime

In [ ]:
# Set up logging to catch errors, etc...

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler(str(time.strftime("%d_%m_%Y")) +"_looker_API_Calls" + ".log")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info('Testing Logs')

# Configure Credentials 


For the purpose of this exercise, we made the credentials available in this python notebook... 


In a production environment, you could setup credentials in a seperate file called looker-config and read in credentials here. 
Sample credentails might look like the following: 
#### Connecting to the Looker API 
[looker-creds]<br/>
api_endpoint: https://<instance>.looker.com:19999/api/3.0 <br/>
login_url: https://<instance>.looker.com:19999/login <br/>
client_id: <Client ID> <br/>
client_secret: <Secret key> <br/>

#### Connecting to a Database
Example creds for Redshift might look like this: <br/>
[redshift] <br/>
username: my_username <br/>
password: my_pwd <br/>
host_name: my_hostname <br/>
port: port_num <br/>
db_name: my_dbname <br/>

# Connect to Looker
LookerAPI class contains information on connecting to Looker via the API. 

The calls available via this api class allow you to 
1. Establish a connection to the API 
2. Get Look Results (run_look)
3. Get Query Results based on Query ID (qid) (run_query)
4. Get Query Results based on Query Slug (run_query_slug)

In [ ]:
class LookerAPI(object):
    """Class that contains methods and variables related to looker API authentication"""
    def __init__(self, api_info):
        self.api_endpoint = api_info['api_endpoint']
        self.client_secret = api_info['client_secret']
        self.client_id = api_info['client_id']
        self.login_endpoint = api_info['login_url']
        print(self.login_endpoint)
        
    def login(self):
        "login to looker API"
        try:
            auth_data = {'client_id':self.client_id, 'client_secret':self.client_secret}
            r = requests.post( self.login_endpoint,data=auth_data) # error handle here
            json_auth = json.loads(r.text)['access_token']
            return json_auth
        except requests.exceptions.RequestException as e:
            logger.error(e)

    def run_look(self, look_id, json_auth,return_format='csv'):
        "run looks"
        try:
            look_run_url = self.api_endpoint + '/looks/{0}/run/{1}'.format(look_id,return_format)
            #r = requests.get(look_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(look_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query(self, query_id, json_auth, return_format='csv'):
        "run query"
        try:
            query_run_url = self.api_endpoint + '/queries/{0}/run/{1}'.format(query_id,return_format)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query_slug(self, query_slug, json_auth):
        "run query"
        try:
            query_slug_run_url = self.api_endpoint + '/queries/slug/{0}'.format(query_slug)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_slug_run_url + '?' + 'access_token=' + json_auth)
            qid=json.loads(r.text)["id"]
            print("Query_id: " + str(qid))
            return LookerAPI.run_query(self, qid, json_auth)
        except requests.exceptions.RequestException as e:
            logger.error(e)

## Test  LookerAPI calls 

Test out API calls on the demonew looker instance! Modify the look ID, Query ID,  or the Query Slug you'd like results from. 

In order to make an API Call, first establish a connection with the Looker Application. Establishing a connection returns a token, that will be used in any subsequent API calls. 

In [8]:
# Set API Credentials for Your Look Instance

creds = {
    'api_endpoint' : 'https://<MY_INSTANCE>.looker.com:19999/api/3.0',
    'login_url': 'https://<MY_INSTANCE>.looker.com:19999/login',
    'client_id' : '<MY_ID>', 
    'client_secret': '<MY_SECRET>'
}

demo = LookerAPI(creds)

# Login to Instance - Return token to be used in subsequent calls.
json_auth = demo.login()

print('Token: ' + json_auth)

# Get Results of Look and Query ID (QID)
look_response = demo.run_look(<MY_LOOK_ID>,json_auth)
print(look_response)

query_response = demo.run_query(<MY_QUERY_ID>, json_auth)
print(query_response)

# # Get Results of Look by passing in the Query Slug. The slug can be found in the URL of any Looker Explore page. 
query_response = demo.run_query_slug("<MY_QUERY_SLUG>", json_auth)
print(query_response)

# Convert Query Response from CSV to a dateframe
records = pd.read_csv(StringIO(query_response))
records.head(5)


SyntaxError: invalid syntax (<ipython-input-8-2a201803f002>, line 18)

We've sucessfully established a connection to Looker that allows us to pull in modeled data from the Looker explore page or a saved look. 